In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Feb__7_19:32:13_PST_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import concept_model
import helper
from utils.log import setup_logger
from utils.ood_utils import run_ood_over_batch
from utils.test_utils import get_measures
# from test_baselines import run_eval

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K


import tensorflow.keras.utils as utils
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.layers as layers

from utils.ood_utils import run_ood_over_batch
from utils.test_utils import get_measures
from utils.stat_utils import multivar_separa 

import os
import argparse
import logging
import numpy as np
import sys
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


physical_devices = tf.config.experimental.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

print(tf.config.experimental.list_physical_devices())

2024-08-12 03:34:39.501885: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 03:34:39.831700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 03:34:39.949938: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 03:34:39.984627: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-12 03:34:40.187849: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
class ARGS:
    def __init__(self):
        self.gpu = "0"
        self.batch_size = 256
        self.epoch = 40
        self.opt = "adam"
        self.thres = 0
        self.val_step = 2
        self.save_step = 1
        self.offset = 0
        self.trained = False
        self.num_concepts = 100

        self.coeff_concept = 10
        self.feat_l2 = True
        self.coeff_feat = 0.1
        self.feat_cosine = False
        self.coeff_cosine = 1
        self.ood = True
        self.score = "energy"
        self.coeff_score = 1 # 
        self.separability = True
        self.coeff_separa = 50

        self.num_hidden = 2

        self.out_data = "MSCOCO" # "augAwA"
        self.temperature_odin = 1000
        self.epsilon_odin = 0.0
        self.temperature_energy = 1
        
        self.name = "AwA2_2_feat_l2_0.1_ood_1_sep_50_rob" # AwA2_baseline, AwA2_feat_l2_0.1_ood_1_sep_50
        self.logdir = "results/"+self.name+"/train_logs"

args = ARGS()

In [4]:
def get_data(bs, ood=True):
    """
    prepare data loaders for ID and OOD data (train/test)
    :param bs: batch size
    :ood: whether to load OOD data as well (False for baseline concept learning by Yeh et al.)
    """

    TRAIN_DIR = "data/AwA2/train"
    VAL_DIR = "data/AwA2/val"
    TEST_DIR = "data/AwA2/test"
    if args.out_data == 'MSCOCO':
        OOD_DIR = "data/MSCOCO"
    elif args.out_data == 'augAwA':
        OOD_DIR = "data/AwA2-train-fractals"

    TARGET_SIZE = (224, 224)
    BATCH_SIZE = bs
    BATCH_SIZE_OOD = bs

    print('Loading images through generators ...')
    train_datagen = ImageDataGenerator(rescale=1. / 255.,
                                       rotation_range=30,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True)
    train_loader = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    batch_size=BATCH_SIZE,
                                                    target_size=TARGET_SIZE,
                                                    class_mode='categorical',
                                                    shuffle=True)

    #print(train_generator.class_indices.items())

    datagen = ImageDataGenerator(rescale=1.0 / 255.)
    val_loader = datagen.flow_from_directory(VAL_DIR,
                                            batch_size=BATCH_SIZE,
                                            target_size=TARGET_SIZE,
                                            class_mode='categorical',
                                            shuffle=False)
    test_loader = datagen.flow_from_directory(TEST_DIR,
                                            batch_size=BATCH_SIZE,
                                            target_size=TARGET_SIZE,
                                            class_mode='categorical',
                                            shuffle=False)
    if ood:
        #numUpdates = int(NUM_TRAIN / BATCH_SIZE) # int(f_train.shape[0] / BATCH_SIZE)
        #NUM_OOD = 31706
        #BATCH_SIZE_OOD = int(NUM_OOD / numUpdates)
        OOD_loader = train_datagen.flow_from_directory(OOD_DIR, #datagen
                                                batch_size=BATCH_SIZE_OOD,
                                                target_size=TARGET_SIZE,
                                                class_mode=None, shuffle=True)
    else:
        OOD_loader = None

    return train_loader, val_loader, test_loader, OOD_loader


def get_class_labels(loader, savepath):
    """
    extract groundtruth class labels from data loader
    :param loader: data loader
    :param savepath: path to the numpy file
    """

    if os.path.exists(savepath):
        y = np.load(savepath)
    else:
        num_data = len(loader.filenames)
        y = []
        for (_, y_batch), _ in zip(loader, range(len(loader))):
            y.extend(y_batch)
       
        np.save(savepath, y)
    return y

def run_eval(feature_model, predict_model, in_loader, out_loader, logger, args, num_classes):
    in_scores = np.array([])
    for i, (x, y) in enumerate(in_loader):
        if i == len(in_loader):
            break
        score = run_ood_over_batch(x, feature_model, predict_model, args, num_classes).numpy()
        in_scores = np.concatenate([in_scores, score])
    out_scores = np.array([])
    for i, x in enumerate(out_loader):
        if i == len(in_loader):
            break
        score = run_ood_over_batch(x, feature_model, predict_model, args, num_classes).numpy()
        out_scores = np.concatenate([out_scores, score])
    in_examples = np.expand_dims(in_scores, axis=1)
    out_examples = np.expand_dims(out_scores, axis=1)
    auroc, aupr_in, aupr_out, fpr, thres95 = get_measures(in_examples, out_examples)
    return in_scores, out_scores, auroc, fpr, thres95

def gaussian_blur(img, kernel_size=11, sigma=5):
    def gauss_kernel(channels, kernel_size, sigma):
        ax = tf.range(-kernel_size // 2 + 1.0, kernel_size // 2 + 1.0)
        xx, yy = tf.meshgrid(ax, ax)
        kernel = tf.exp(-(xx ** 2 + yy ** 2) / (2.0 * sigma ** 2))
        kernel = kernel / tf.reduce_sum(kernel)
        kernel = tf.tile(kernel[..., tf.newaxis], [1, 1, channels])
        return kernel
    
    gaussian_kernel = gauss_kernel(tf.shape(img)[-1], kernel_size, sigma)
    gaussian_kernel = gaussian_kernel[..., tf.newaxis]
    
    return tf.nn.depthwise_conv2d(np.expand_dims(img, axis=0), gaussian_kernel, [1, 1, 1, 1],
                                  padding='SAME', data_format='NHWC')


In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

#if not os.path.exists(args.output_dir):
#    os.makedirs(args.output_dir)

if args.separability:
    args.ood = True
USE_OOD = args.ood
BATCH_SIZE = args.batch_size
EPOCH = args.epoch
THRESHOLD = args.thres
trained = args.trained
N_CONCEPT = args.num_concepts
offset = args.offset
topic_modelpath = os.path.join(args.logdir, args.name,'topic_epoch{}.weights.h5'.format(offset))
#topic_modelpath = os.path.join(args.logdir, args.name,'topic_latest.h5')
topic_savepath = os.path.join(args.logdir, args.name,'topic_vec_inceptionv3.npy')

logger = setup_logger(args)

train_loader, val_loader, test_loader, ood_loader =  get_data(BATCH_SIZE, ood=USE_OOD)

#print(train_generator.class_indices.items())
#assert ('_OOD', 0) in val_generator.class_indices.items()
#y_train = get_class_labels(train_loader, savepath='data/Animals_with_Attributes2/y_train.npy')
y_val = get_class_labels(val_loader, savepath='data/AwA2/y_val.npy')
y_test = get_class_labels(test_loader, savepath='data/AwA2/y_test.npy')

# preds_cls_idx = y_test.argmax(axis=-1)
# idx_to_cls = {v: k for k, v in test_generator.class_indices.items()}
# preds_cls = np.vectorize(idx_to_cls.get)(preds_cls_idx)
# filenames_to_cls = list(zip(test_generator.filenames, preds_cls))


# Loads model
feature_model, predict_model = helper.load_model_inception_new(train_loader, val_loader, \
           batch_size=BATCH_SIZE, input_size=(224,224), pretrain=True, \
           modelname='./results/AwA2/inceptionv3_AwA2_rob_ep_49.weights.h5', split_idx=-5)

2024-08-12 03:34:51,459 [INFO] utils.log: <__main__.ARGS object at 0x7f1a74030610>


Loading images through generators ...
Found 29841 images belonging to 50 classes.
Found 3709 images belonging to 50 classes.
Found 3772 images belonging to 50 classes.
Found 40670 images belonging to 1 classes.


2024-08-12 03:34:55.836555: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:af:00.0, compute capability: 7.0




original model to be trained


/home/dingw/.conda/envs/adv_train/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/dingw/.conda/envs/adv_train/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1723426518.041243 1257438 service.cc:146] XLA service 0x7f19c40038e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723426518.041327 1257438 service.cc:154]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-08-12 03

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_2 (InputLayer)            │ (None, 5, 5, 2048)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d            │ (None, 2048)           │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │        12,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 537,394 (2.05 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 537,394 (2.05 MB)

None


In [6]:
## Concept Learning
x, _ = test_loader.__next__()
f = feature_model(x[:10])
# topic model: intermediate feature --> concept score --> recovered feature --> prediction (50 classes)
topic_model_pr = concept_model.TopicModel(f, N_CONCEPT, THRESHOLD, predict_model, args.num_hidden)
_ = topic_model_pr(f)
print(topic_model_pr.build_graph(f).summary())

if os.path.exists(topic_modelpath):
    topic_model_pr.load_weights(topic_modelpath)
    logger.info(f'topic model loaded from {topic_modelpath}')

W0000 00:00:1723426611.044710 1257152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723426611.095545 1257152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723426611.095875 1257152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723426611.096172 1257152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723426611.096461 1257152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723426611.096761 1257152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723426611.097063 1257152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723426611.097374 1257152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723426611.097689 1257152 gp

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ f_input             │ (None, 5, 5,      │          0 │ -                 │
│ (InputLayer)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_8 (Lambda)   │ (None, 5, 5,      │          0 │ f_input[0][0]     │
│                     │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ weight (Weight)     │ (2048, 100)       │    204,800 │ lambda_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mat_mul (MatMul)    │ (None, 5, 5,      │          0 │ f_input[0][0],    │
│                     │ 2048)             │            │ weight[0][0],     │
│                     │                   │            │ lambda_8[0][0],   │
│                     │                   │            │ weight[0][0],     │
│                     │                   │            │ lambda_11[0][0],  │
│                     │                   │            │ weight_1[0][0],   │
│                     │                   │            │ re_lu[0][0],      │
│                     │                   │            │ weight_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_9 (Lambda)   │ (None, 5, 5, 100) │          0 │ mat_mul[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 5, 5, 100) │          0 │ mat_mul[0][0],    │
│                     │                   │            │ lambda_9[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_10 (Lambda)  │ (None, 5, 5, 1)   │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_11 (Lambda)  │ (None, 5, 5, 100) │          0 │ multiply[0][0],   │
│                     │                   │            │ lambda_10[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ weight_1 (Weight)   │ (100, 500)        │     50,000 │ f_input[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 5, 5, 500) │          0 │ mat_mul[2][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ weight_2 (Weight)   │ (500, 2048)       │  1,024,000 │ f_input[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 50)        │    537,394 │ mat_mul[3][0]     │
│ (Functional)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,816,194 (6.93 MB)

 Trainable params: 1,278,800 (4.88 MB)

 Non-trainable params: 537,394 (2.05 MB)

None


In [ ]:
## Concept Learning

if args.opt =='sgd':
    """
    optimizer = SGD(lr=0.1)
    optimizer_state = [optimizer.iterations, optimizer.lr, optimizer.momentum, optimizer.decay]
    optimizer_reset = tf.compat.v1.variables_initializer(optimizer_state)
    """
    optimizer = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
elif args.opt =='adam':
    optimizer = Adam(learning_rate=0.01)
    optimizer_state = [optimizer.iterations, optimizer.learning_rate, optimizer.beta_1, optimizer.beta_2, optimizer.weight_decay]
    optimizer_reset = tf.compat.v1.variables_initializer(optimizer_state)

train_acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.CategoricalAccuracy()
test_acc_metric = keras.metrics.CategoricalAccuracy()
softmax = layers.Activation('softmax')

@tf.function
def train_step(x_in, y_in, x_out=None, thres=None):
    #tf.keras.applications.inception_v3.preprocess_input(x_in)
    f_in = feature_model(x_in)
    f_in_n = K.l2_normalize(f_in,axis=(3))


    obj_terms = {} # terms in the objective function
    COEFF_CONCEPT = args.coeff_concept #10 -> 5 -> 1 
    with tf.GradientTape() as tape:
        f_in_recov, logits_in, topic_vec_n = topic_model_pr(f_in, training=True)
        pred_in = softmax(logits_in) # class prediction using concept scores
        topic_prob_in_n = K.dot(f_in_n, topic_vec_n) # normalized concept scores

        # total loss
        CE_IN = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_in, pred_in))
        loss_coherency = tf.reduce_mean(tf.nn.top_k(K.transpose(K.reshape(topic_prob_in_n,(-1,N_CONCEPT))),k=10,sorted=True).values)
        loss_similarity = tf.reduce_mean(K.dot(K.transpose(topic_vec_n), topic_vec_n) - tf.eye(N_CONCEPT))
        loss = CE_IN - COEFF_CONCEPT*loss_coherency + COEFF_CONCEPT*loss_similarity  # baseline: Yeh et al.
        obj_terms['[ID] CE'] = CE_IN
        obj_terms['[ID] concept coherency'] = loss_coherency
        obj_terms['[ID] concept similarity'] = loss_similarity
        #print('y_in: '+type(y_in).__name__)
        #print('pred_in: '+type(pred_in).__name__)
        #print('CE_IN: '+type(CE_IN).__name__)
        #print('loss coher: '+type(loss_coherency).__name__)
        #print('loss_sim: '+type(loss_similarity).__name__)
        #print('loss: '+type(loss).__name__)
        
        if args.feat_l2:
            loss_l2 = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.pow(f_in-f_in_recov,2), axis=(1,2,3))))
            #loss_l2 = tf.reduce_mean(tf.reduce_sum(tf.pow(f_in-f_in_recov,2), axis=(1,2,3)))
            loss += args.coeff_feat*loss_l2 #0.07, 0.02
            obj_terms['feature L2'] = loss_l2

        if args.feat_cosine:
            loss_cosine = tf.reduce_mean(tf.keras.losses.cosine_similarity(f_in, f_in_recov)) # equivalent to: tf.reduce_mean(tf.reduce_sum(tf.math.multiply(f_in, f_in_recov),axis=(1,2,3))/(tf.sqrt(tf.reduce_sum(tf.pow(f_in,2),axis=(1,2,3)))*tf.sqrt(tf.reduce_sum(tf.pow(f_in_recov,2),axis=(1,2,3)))))
            loss_cosine = 1 - loss_cosine # cosine distance, range=[0, 2]
            loss += args.coeff_cosine*loss_cosine
            obj_terms['feature cosine distance'] = loss_cosine
        
        if args.score:
            s_in = run_ood_over_batch(x_in, feature_model, predict_model, args, num_classes=50)
            s_out = run_ood_over_batch(x_out, feature_model, predict_model, args, num_classes=50)

            if args.coeff_score > 0.0:
                # scores from OOD detector when using recovered features
                s_in_recov = run_ood_over_batch(x_in, feature_model, topic_model_pr, args, num_classes=50)
                s_out_recov = run_ood_over_batch(x_out, feature_model, topic_model_pr, args, num_classes=50)

                s_original = tf.concat((s_in, s_out), axis=0)
                s_recovered = tf.concat((s_in_recov, s_out_recov), axis=0)
                loss_score = tf.reduce_mean(tf.pow(s_original - s_recovered, 2))
                loss += args.coeff_score*loss_score
                obj_terms['score difference'] = loss_score

                """
                # Debugging
                auroc, aupr_in, aupr_out, fpr95, thres95 = get_measures(s_in.numpy()[:,None], s_out.numpy()[:,None])
                print(f'auroc: {auroc}, aupr in: {aupr_in}, aupr out: {aupr_out}, fpr95: {fpr95}')
                auroc, aupr_in, aupr_out, fpr95, thres95 = get_measures(s_in_recov.numpy()[:,None], s_out_rec
ov.numpy()[:,None])
                print(f'auroc: {auroc}, aupr in: {aupr_in}, aupr out: {aupr_out}, fpr95: {fpr95}')
                input()
                """
        
        if args.separability:
            f_out = feature_model(x_out)
            f_out_n = K.l2_normalize(f_out,axis=(3))
            _, logits_out, _ = topic_model_pr(f_out, training=True)
            #tf.debugging.assert_equal(topic_vec_n, topic_vec_n_out) 
            topic_prob_out_n = K.dot(f_out_n, topic_vec_n)
            

            # max --> smoothly approximated by logsumexp
            #T = tf.Variable(1e+3, dtype=tf.float32)
            T = 1e+3
            prob_max_in = 1/T*tf.math.reduce_logsumexp(T*topic_prob_in_n,axis=(1,2))
            prob_min_in = -1/T*tf.math.reduce_logsumexp(-T*topic_prob_in_n,axis=(1,2))

            ## concept scores of "true" ID set and "true" OOD set
            concept_in_true = tf.where(tf.abs(prob_max_in) > tf.abs(prob_min_in), prob_max_in, prob_min_in)
            prob_max_out = 1/T*tf.math.reduce_logsumexp(T*topic_prob_out_n,axis=(1,2))
            prob_min_out = -1/T*tf.math.reduce_logsumexp(-T*topic_prob_out_n,axis=(1,2))
            concept_out_true = tf.where(tf.abs(prob_max_out) > tf.abs(prob_min_out), prob_max_out, prob_min_out)
            
            ## concept scores of "detected" ID set and "detected" OOD set
            concept_in = tf.concat([concept_in_true[s_in>=thres], concept_out_true[s_out>=thres]], axis=0) 
            concept_out = tf.concat([concept_in_true[s_in<thres], concept_out_true[s_out<thres]], axis=0)

            # global separability
            loss_separa = multivar_separa(concept_in, concept_out)
            loss -= args.coeff_separa*loss_separa
            obj_terms['separability'] = loss_separa

    obj_terms['total loss.......'] = loss
    train_acc_metric.update_state(y_in, logits_in)
    #print(obj_terms)

    # calculate the gradients using our tape and then update the model weights
    grads = tape.gradient(loss, topic_model_pr.trainable_variables)
    optimizer.apply_gradients(zip(grads, topic_model_pr.trainable_variables))
    #print(type(loss).__name__, ":", grads)
    #input()
    return obj_terms

if not trained:
    for layer in topic_model_pr.layers[:-1]:
        #print(layer.trainable)
        layer.trainable = True

    # check all weights are included in trainable_variables
    # for i, var in enumerate(topic_model_pr.trainable_variables):
    #     print(topic_model_pr.trainable_variables[i].name)


    if args.score and args.separability: # identify threshold from held-out set
        datagen = ImageDataGenerator(rescale=1.0 / 255.)
        if args.out_data == 'MSCOCO':
            out_gen = datagen.flow_from_directory('data/MSCOCO/test',batch_size=150,target_size=(224,224),class_mode=None,shuffle=False)
        elif args.out_data == 'augAwA':
            out_gen = datagen.flow_from_directory('data/AwA2-test-fractals',batch_size=150,target_size=(224,224),class_mode=None,shuffle=False)
        _, _, _, _, thres = run_eval(feature_model, predict_model, val_loader, out_gen, logger, args, 50)
        thres = float(thres)
    else:
        thres = None

    df_obj_terms = pd.DataFrame()
    for epoch in range(offset+1, offset+EPOCH+1):
        logger.info(f"\n[INFO] starting epoch {epoch}/{offset+EPOCH} ---------------------------------")
        sys.stdout.flush()
        epochStart = time.time()
        
        for step, (x_in, y_in) in enumerate(train_loader):
            
            s = np.random.uniform(0, 1, size=6) > 0.5
            if s[4]:
              sigma = np.random.uniform(0.2, 0.8, size=len(x_in))
              for i_, sig in enumerate(sigma):
                x_in[i_] = gaussian_blur(x_in[i_], kernel_size=10, sigma=sig)
            if s[0]:
              x_in= tf.image.random_hue(x_in, 0.1, seed=None)
            if s[1]:
              x_in = tf.image.random_brightness(x_in, 0.3, seed=None)
            if s[2]:
              x_in = tf.image.random_contrast(x_in, 0.7, 1)
            if s[3]:
              saturation_factor = np.random.uniform(0.5, 1)
              x_in = tf.image.adjust_saturation(x_in, saturation_factor)
            
            if s[5]:
              x_in += np.random.uniform(0, 0.1, size=x_in.shape)
            
            step += 1 # starts from 1
            if step > len(train_loader):
                break

            if USE_OOD:
                x_out = ood_loader.__next__()
                
                s = np.random.uniform(0, 1, size=6) > 0.5
                if s[4]:
                  sigma = np.random.uniform(0.2, 0.8, size=len(x_out))
                  for i_, sig in enumerate(sigma):
                    x_out[i_] = gaussian_blur(x_out[i_], kernel_size=10, sigma=sig)
                if s[0]:
                  x_out = tf.image.random_hue(x_out, 0.1, seed=None)
                if s[1]:
                  x_out = tf.image.random_brightness(x_out, 0.3, seed=None)
                if s[2]:
                  x_out = tf.image.random_contrast(x_out, 0.7, 1)
                if s[3]:
                  saturation_factor = np.random.uniform(0.5, 1)
                  x_out = tf.image.adjust_saturation(x_out, saturation_factor)
                
                if s[5]:
                  x_out += np.random.uniform(0, 0.1, size=x_out.shape)
                
                obj_terms = train_step(x_in, y_in, x_out, thres)
            else:
                obj_terms = train_step(x_in, y_in)

            # Log every 50 batches
            if step % 20 == 0:
                #print(topic_model_pr.layers[0].get_weights()[0])
                for term in obj_terms:
                    logger.info(f'[STEP{step}] {term}: {obj_terms[term]}')
            for term in obj_terms:
                obj_terms[term] = obj_terms[term].numpy()
            obj_terms["epoch"] = epoch
            obj_terms["step"] = step
            df_obj = pd.Series(obj_terms)
            df_obj_terms = pd.concat([df_obj_terms, pd.DataFrame(df_obj).T], axis=0)
        
        train_acc = train_acc_metric.result()
        logger.info("Training acc over epoch: %.4f" % (float(train_acc),))
        
        # show timing information for the epoch
        epochEnd = time.time()
        elapsed = (epochEnd - epochStart) / 60.0
        logger.info("Time taken: %.2f minutes" % (elapsed))

        df_obj_terms = df_obj_terms.reset_index(drop=True)
        df_obj_terms_melt = pd.melt(df_obj_terms, id_vars=["epoch", "step"], 
                                    value_vars=[col for col in df_obj_terms.columns if col in 
                                                ['[ID] CE', '[ID] concept coherency', 'feature L2', 
                                                 '[ID] concept similarity', 'ood score difference', 
                                                 'id & ood separability', 'total loss']],
                                    var_name="loss_term", value_name="loss_value")

        plt.figure()
        sns.lineplot(data=df_obj_terms_melt, x="epoch", y="loss_value", hue="loss_term")
        plt.savefig(args.logdir+"/train_loss.png")
        plt.close()
        plt.figure()
        sns.lineplot(data=df_obj_terms_melt[(df_obj_terms_melt["loss_term"]=='[ID] CE') | (df_obj_terms_melt["loss_term"]=='[ID] concept coherency') | 
                                            (df_obj_terms_melt["loss_term"]=='[ID] concept similarity')], 
                     x="epoch", y="loss_value", hue="loss_term")
        plt.savefig(args.logdir+"/train_loss1.png")
        plt.close()


        # Reset training metrics at the end of each epoch
        train_acc_metric.reset_state()
        if epoch % args.save_step == 0:
            topic_model_pr.save_weights(os.path.join(args.logdir, args.name,'topic_epoch{}.weights.h5'.format(epoch)))

        if epoch % args.val_step == 0:
            _, logits_val, _ = topic_model_pr(feature_model.predict(val_loader), training=False)
            pred_val = softmax(logits_val)
            val_acc_metric.update_state(y_val, logits_val)
            val_acc = val_acc_metric.result()
            logger.info("[EPOCH %d] Validation acc: %.4f" % (epoch, float(val_acc)))
            val_acc_metric.reset_state()
            del logits_val
        
        logger.flush()


topic_vec = topic_model_pr.layers[0].get_weights()[0]   # 1, (2048, num_concepts)
# recov_vec = topic_model_pr.layers[-3].get_weights()[0]
topic_vec_n = topic_vec/(np.linalg.norm(topic_vec,axis=0,keepdims=True)+1e-9)
np.save(topic_savepath,topic_vec)
# np.save('results/Animals_with_Attributes2_energy_COCO/recov_vec_inceptionv3.npy',recov_vec)

assert np.shape(topic_vec)[1] == N_CONCEPT
# topic_model_pr.evaluate(f_test, y_test)
# f_val_recovered = topic_model_pr.predict(f_val)


f_test = feature_model.predict(test_loader)
_, logits_test, _ = topic_model_pr(f_test, training=False)
pred_test = softmax(logits_test)
test_acc_metric.update_state(y_test, logits_test)
test_acc = test_acc_metric.result()
logger.info('[ID TEST] Accuracy of topic model on test set: %f' %test_acc)

logger.flush()

Found 40670 images belonging to 1 classes.


2024-08-12 03:54:01,747 [INFO] utils.log: 
[INFO] starting epoch 1/40 ---------------------------------


2023
Cause: Unable to locate the source code of <function lower at 0x7f1afe533e20>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-08-12 03:54:18,625 [WARNING] tensorflow: AutoGraph could not transform <function lower at 0x7f1afe533e20> and will run it as-is.
Cause: Unable to locate the source code of <function lower at 0x7f1afe533e20>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function lower at 0x7f1afe533e20>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

2024-08-12 09:05:13,358 [INFO] utils.log: [EPOCH 10] Validation acc: 0.8471
2024-08-12 09:05:13,361 [INFO] utils.log: 
[INFO] starting epoch 11/40 ---------------------------------
2024-08-12 09:10:23,527 [INFO] utils.log: [STEP20] [ID] CE: 0.8701656460762024
2024-08-12 09:10:23,530 [INFO] utils.log: [STEP20] [ID] concept coherency: 0.5915206074714661
2024-08-12 09:10:23,532 [INFO] utils.log: [STEP20] [ID] concept similarity: 0.007626138161867857
2024-08-12 09:10:23,534 [INFO] utils.log: [STEP20] feature L2: 159.4639892578125
2024-08-12 09:10:23,536 [INFO] utils.log: [STEP20] score difference: 0.4723358154296875
2024-08-12 09:10:23,538 [INFO] utils.log: [STEP20] separability: 0.024843793362379074
2024-08-12 09:10:23,541 [INFO] utils.log: [STEP20] total loss.......: 10.207765579223633
2024-08-12 09:15:37,303 [INFO] utils.log: [STEP40] [ID] CE: 0.818822979927063
2024-08-12 09:15:37,308 [INFO] utils.log: [STEP40] [ID] concept coherency: 0.594336986541748
2024-08-12 09:15:37,310 [INFO] uti

15/15 ━━━━━━━━━━━━━━━━━━━━ 71s 5s/step


2024-08-12 10:07:02,242 [INFO] utils.log: [EPOCH 12] Validation acc: 0.8485
2024-08-12 10:07:02,245 [INFO] utils.log: 
[INFO] starting epoch 13/40 ---------------------------------
2024-08-12 10:12:41,729 [INFO] utils.log: [STEP20] [ID] CE: 0.8067196607589722
2024-08-12 10:12:41,732 [INFO] utils.log: [STEP20] [ID] concept coherency: 0.5939410328865051
2024-08-12 10:12:41,734 [INFO] utils.log: [STEP20] [ID] concept similarity: 0.008377769030630589
2024-08-12 10:12:41,736 [INFO] utils.log: [STEP20] feature L2: 156.13995361328125
2024-08-12 10:12:41,738 [INFO] utils.log: [STEP20] score difference: 0.5526642799377441
2024-08-12 10:12:41,740 [INFO] utils.log: [STEP20] separability: 0.02860177494585514
2024-08-12 10:12:41,742 [INFO] utils.log: [STEP20] total loss.......: 9.687657356262207
2024-08-12 10:18:03,918 [INFO] utils.log: [STEP40] [ID] CE: 0.8529496788978577
2024-08-12 10:18:03,922 [INFO] utils.log: [STEP40] [ID] concept coherency: 0.6044558882713318
2024-08-12 10:18:03,924 [INFO] ut

15/15 ━━━━━━━━━━━━━━━━━━━━ 77s 5s/step


2024-08-12 11:13:12,422 [INFO] utils.log: [EPOCH 14] Validation acc: 0.8579
2024-08-12 11:13:12,425 [INFO] utils.log: 
[INFO] starting epoch 15/40 ---------------------------------
2024-08-12 11:18:33,043 [INFO] utils.log: [STEP20] [ID] CE: 0.8518707752227783
2024-08-12 11:18:33,047 [INFO] utils.log: [STEP20] [ID] concept coherency: 0.5886412262916565
2024-08-12 11:18:33,049 [INFO] utils.log: [STEP20] [ID] concept similarity: 0.008842642419040203
2024-08-12 11:18:33,051 [INFO] utils.log: [STEP20] feature L2: 154.8707275390625
2024-08-12 11:18:33,053 [INFO] utils.log: [STEP20] score difference: 0.47257310152053833
2024-08-12 11:18:33,055 [INFO] utils.log: [STEP20] separability: 0.027722885832190514
2024-08-12 11:18:33,057 [INFO] utils.log: [STEP20] total loss.......: 9.627387046813965
2024-08-12 11:24:05,063 [INFO] utils.log: [STEP40] [ID] CE: 1.0045757293701172
2024-08-12 11:24:05,084 [INFO] utils.log: [STEP40] [ID] concept coherency: 0.5862016677856445
2024-08-12 11:24:05,087 [INFO] u